# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

#BE CAREFUL OF API USAGE

#Import API key
from config import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
#export csv
# change from cities csv to weather csv
weathers_df = pd.read_csv('output_data/weather_data.csv')

#show csv
weathers_df.head(10)

,Unnamed: 0,City,Country,Lat,Lng,Max Temperature,Humidity,Wind speed,Date,Cloudiness
0,0,rikitea,PF,-23.1203,-134.9692,27.12,75.0,4.22,1.615010e+09,37.0
1,1,chichester,GB,50.8367,-0.7800,0.53,90.0,0.45,1.615010e+09,5.0
2,2,cooma,AU,-36.2333,149.1333,20.00,28.0,3.60,1.615010e+09,0.0
3,3,qaanaaq,GL,77.4840,-69.3632,-26.77,67.0,2.75,1.615010e+09,0.0
4,4,puerto carreno,CO,6.1890,-67.4859,26.13,80.0,2.24,1.615010e+09,94.0
5,5,hobart,AU,-42.8794,147.3294,16.71,55.0,7.72,1.615010e+09,0.0
6,6,tuktoyaktuk,CA,69.4541,-133.0374,-24.00,77.0,3.60,1.615010e+09,1.0
7,7,east london,ZA,-33.0153,27.9116,20.00,83.0,3.60,1.615010e+09,0.0
8,8,ribeira grande,PT,38.5167,-28.7000,15.42,84.0,6.05,1.615010e+09,86.0
9,9,tahta,EG,26.7693,31.5021,9.47,53.0,2.06,1.615010e+09,0.0


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
#configure gmaps
gmaps.configure(api_key=g_key)

In [4]:
#here use lat and long as locations and humidity as weight, set to type float

location = weathers_df[["Lat", "Lng"]].astype(float)
humidity_x = weathers_df["Humidity"].astype(float)


In [ ]:
#we create a heatmap
fig = gmaps.figure()
#heat_layer
heat_layer = gmaps.heatmap_layer(location, weights= humidity_x, dissipating = False, 
                                 max_intensity = 100, point_radius= 1)
fig.add_layer(heat_layer)
#show figure
fig


### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:

hotel_df = weathers_df[(weathers_df['Cloudiness'] == 0) & (weathers_df['Max Temperature']< 70)& (weathers_df['Wind speed']< 9)]
hotel_df.head(10)

#dropping null values
hotel_df = hotel_df.dropna()
hotel_df

,Unnamed: 0,City,Country,Lat,Lng,Max Temperature,Humidity,Wind speed,Date,Cloudiness
2,2,cooma,AU,-36.2333,149.1333,20.00,28.0,3.60,1.615010e+09,0.0
3,3,qaanaaq,GL,77.4840,-69.3632,-26.77,67.0,2.75,1.615010e+09,0.0
5,5,hobart,AU,-42.8794,147.3294,16.71,55.0,7.72,1.615010e+09,0.0
7,7,east london,ZA,-33.0153,27.9116,20.00,83.0,3.60,1.615010e+09,0.0
9,9,tahta,EG,26.7693,31.5021,9.47,53.0,2.06,1.615010e+09,0.0
...,...,...,...,...,...,...,...,...,...,...
536,536,whitehorse,LY,29.0331,21.5482,14.12,36.0,6.93,1.615011e+09,0.0
540,540,jinchang,SV,13.5928,-89.8275,24.60,65.0,3.00,1.615011e+09,0.0
541,541,port hedland,CL,-29.9533,-71.3436,16.00,94.0,0.81,1.615010e+09,0.0
543,543,edeia,BR,-20.3297,-40.2925,24.70,78.0,5.66,1.615011e+09,0.0


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:

hotel_df['Hotel Name'] = ""

hotel_df.head()


,Unnamed: 0,City,Country,Lat,Lng,Max Temperature,Humidity,Wind speed,Date,Cloudiness,Hotel Name
2,2,cooma,AU,-36.2333,149.1333,20.00,28.0,3.60,1.615010e+09,0.0,
3,3,qaanaaq,GL,77.4840,-69.3632,-26.77,67.0,2.75,1.615010e+09,0.0,
5,5,hobart,AU,-42.8794,147.3294,16.71,55.0,7.72,1.615010e+09,0.0,
7,7,east london,ZA,-33.0153,27.9116,20.00,83.0,3.60,1.615010e+09,0.0,
9,9,tahta,EG,26.7693,31.5021,9.47,53.0,2.06,1.615010e+09,0.0,


In [10]:
#Parameters that needed to be included are 
#radius 5000,
#type lodging, 

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "radius" : 5000,
    "type": "lodging",
    "key": g_key,
}
#interrows 
for index, row in hotel_df.iterrows():

#City
    hotel_city = row['City']
    params['keyword'] = hotel_city

    print(f"Getting Results for Index {index}: {hotel_city}.")
  
    
    response = requests.get(base_url, params=params).json()
    
    results = response['results']
    
    try:
        print(f"Nearby {hotel_city} hotel is {results[0]['Hotel Name']}.")
        
        hotel_df.loc[index, 'Hotel Name'] = results[0]['Hotel Name']

   
        
    except (KeyError, IndexError):
        print("Missing field....")
        

Getting Results for Index 2: cooma.
Missing field....
Getting Results for Index 3: qaanaaq.
Missing field....
Getting Results for Index 5: hobart.
Missing field....
Getting Results for Index 7: east london.
Missing field....
Getting Results for Index 9: tahta.
Missing field....
Getting Results for Index 13: gat.
Missing field....
Getting Results for Index 18: port alfred.
Missing field....
Getting Results for Index 36: raypur.
Missing field....
Getting Results for Index 45: sterling.
Missing field....
Getting Results for Index 48: tasiilaq.
Missing field....
Getting Results for Index 53: mahebourg.
Missing field....
Getting Results for Index 55: cozumel.
Missing field....
Getting Results for Index 57: upernavik.
Missing field....
Getting Results for Index 62: cape town.
Missing field....
Getting Results for Index 69: longyearbyen.
Missing field....
Getting Results for Index 83: illoqqortoormiut.
Missing field....
Getting Results for Index 92: odweyne.
Missing field....
Getting Results 

In [ ]:
#display results
hotel_df

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]


In [ ]:
#show heatmap fig
